# АНАЛИЗ ТЕКСТОВЫХ ДАННЫХ И ПОНЯТИЙ НА ОСНОВЕ КАТЕГОРИЙ ВИКИПЕДИИ / ANALYSIS OF TEXT DATA AND TERMS BASED ON WIKIPEDIA CATEGORIES

Информация о категориях позволяет выделять общие черты предметов и явлений и использовать это для объединения и общего использования знаний, статистики.
Если появляется новый предмет ранее известной категории, то для него сразу становится известны (или достаточно вероятны) многие характеристики по факту принадлежности к этой категории, что можно использовать для создания гипотез, ускоренного обучения или обработки неизвестных вещей. Если какое-то явление происходит с разными объектами категории, то возможно это происходит и со всей категорией и эту гипотезу можно проверить, что было бы менее очевидно при рассмотрени объектов по отдельности. Группируя статистику по категориям можно получить более гибкие результаты, чем по отдельным объектам. <br> <br>
Например если известно про посуду, что она используется как ёмкость для употребления пищи, может быть сделана из хрупких материалов, то это заранее вероятно относится и к новому вид посуды, о котором в данный момент известно только название и то что это является посудой. Если в тексте употребляются слова "кружка", "тарелка". "чаша", то это может быть объёдинено в "посуда", что будет содержать гораздо более выделяющееся число упоминаний, чем эти слова по отдельности.
___

Information about categories allows to recognize common features of objects and phenomena, which can be used for shared use of knowledge and statistics. If some new object appears which is related to a previously known category, this relation tells in advance about highly probably features of the object. This allows to create new hypotheses to check, speed up training or process previously unknown things. If something happens with several objects of a category, it is possible that it happens with every object of the category and this hypothesis can be checked, while this hypothesis could be less obvoious in separate considerations of single objects. Statistics aggregated by categories can be more agile than aggregated by separate objects. <br> <br>
For example, if it is known about crockery that it is used as a container for consuming meals, can be made of fragile materials, the same information is related in advance to some new type of crockery for which nothing is yet known, except for its title and belonging to the "crockery" category. If some text has occurences of words like "cup", "dish", "bowl" it can be united into "crockery" category with a much more significant number of occurences than each single word.

### Структура / Structure

Проекты, описанные далее, не зависят друг от друга, но используют категории из Википедии, загружаемые в коде пункта "ИЗВЛЕЧЕНИЕ КАТЕГОРИЙ ВИКИПЕДИИ / WIKIPEDIA CATEGORIES EXTRACTION". Далее в проектах осуществляется подготовка данных, обработка и отображение результатов с выводами.  Если указана папка в Гугл Диске, то категории Википедии и получаемые данные записываются туда и копируются оттуда для экономии времени (если не указан параметр "OVEWRITE").
___

Projects below are not interconnected but use Wikipedia categories obtained by the code in "ИЗВЛЕЧЕНИЕ КАТЕГОРИЙ ВИКИПЕДИИ / WIKIPEDIA CATEGORIES EXTRACTION" chapter. After that, each project consists of data preparation, processing and result demonstration with conclusions. If a Google Drive directory is specified, the categories and processed data are written there and copied from there to save time (if "OVERWRITE" parameter is not set).

## 0) ИЗВЛЕЧЕНИЕ КАТЕГОРИЙ ВИКИПЕДИИ / WIKIPEDIA CATEGORIES EXTRACTION

Категории Википедии взяты из https://dumps.wikimedia.org/. Для удобства они могут сохраняться в указанную в коде папку **USED_GOOGLE_DRIVE_DIR** в Google Drive. Если дампы были скачаны ранее, то скачанные файлы копируются в рабочую папку.
Исходные данные занимают примерно 5Гб на диске. После их обработки создаются данные примерно на 1Гб, а исходные данные по категориям могут быть удалены.
___

Wikipedia categories are taken from https://dumps.wikimedia.org/. For convinience, they can be saved in specified in the code directory **USED_GOOGLE_DRIVE_DIR** on Google Drive. If these dumps were downloaded before, these files are copied into the working directory.
The raw data takes approximately 5GB on the disk. After processing of these files, data of approximately 1GB is created, and the initial data for categories can be removed.

In [1]:
import numpy as np
import pickle
import os
import gc
import random
from tqdm import tqdm
import re
import json
import ipywidgets as widgets
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

USED_GOOGLE_DRIVE_DIR = "" #"/content/drive/MyDrive/wiki_cats/"
if USED_GOOGLE_DRIVE_DIR != "":
  from google.colab import drive,files
  drive.mount('/content/drive')
  !mkdir -p $USED_GOOGLE_DRIVE_DIR
  !cp {USED_GOOGLE_DRIVE_DIR}/*.* ./

In [2]:
%%time
#datasets to download and use
wiki = "ruwiki"
version = "20240201" #"20221220" #"latest"
data = ["page","categorylinks"]

OVERWRITE = False #parameter to overwrite existing files
#download and extract archives if files don't already exist or manual overwrite is set
for datafile in data:
  if OVERWRITE or (not os.path.exists(datafile + ".pickle")) and (not os.path.exists(wiki + version + datafile + ".sql")):
    !wget https://dumps.wikimedia.org/{wiki}/{version}/{wiki}-{version}-{datafile}.sql.gz
    !gzip -d {wiki}-{version}-{datafile}.sql.gz

--2024-02-21 07:37:45--  https://dumps.wikimedia.org/ruwiki/20240201/ruwiki-20240201-page.sql.gz
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.71, 2620:0:861:3:208:80:154:71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 287715110 (274M) [application/octet-stream]
Saving to: ‘ruwiki-20240201-page.sql.gz’

ruwiki-20240201-pag 100%[===================>] 274.39M  4.55MB/s    in 59s     

2024-02-21 07:38:44 (4.65 MB/s) - ‘ruwiki-20240201-page.sql.gz’ saved [287715110/287715110]

--2024-02-21 07:38:56--  https://dumps.wikimedia.org/ruwiki/20240201/ruwiki-20240201-categorylinks.sql.gz
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.71, 2620:0:861:3:208:80:154:71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 584486374 (557M) [application/octet-stream]
Saving to: 

Строки дампов парсятся для извлечения пар **ID** - **Название страницы** и для извлечения пар **Родительская категория** - **Дочерняя категория**. Полученные данные сохраняются в файлы. Если парсинг был произведён ранее, то данные считываются из файлов.
___

Dump rows are parsed to extract **ID** - **Page title** pairs and **Parent category** - **Son category** pairs. Obtained results are saved into files. If the parsing was done before, the results are loaded from the files.

In [3]:
%%time

gc.collect()

def splitLineIntoItems(st):
    #split Wikipedia dump file line into separate items in brackets, each with list of values, separeted by commas
    result = [] #list of lists to return
    pieceNow = "" #current string in brackets
    quote = False #current state in quote
    quote2 = False #current state in single quote
    openned = False #current state in brackets
    ix = 0 #current char index
    for ix in range(len(st)):
        if not openned and st[ix] == "(": #open brackets
            openned = True
            continue
        if openned and (not quote) and (not quote2) and st[ix] == ")": #close brackets and save data
            vals = pieceNow.split(",") #split current string by commas to get values list
            for i in range(len(vals)):
              vals[i] = vals[i].replace('$%$%', ',') #restore hidden commas
            result.append(vals) #add item in brackets to result
            pieceNow = "" #reset state
            openned = False
            quote = False
            quote2 = False
            continue
        if openned:
            if st[ix] == "," and (quote or quote2):
                pieceNow += '$%$%' #replace inner commas by special string to prevent from split
            else:
                pieceNow += st[ix] #add plain char to current piece
        backs = 0
        while st[ix - backs - 1] == "\\": #get current backslash amount. Odd amount means special char (\' or \")
            backs += 1
        if st[ix] == '"' and (not quote) and (backs%2 == 0): #close quotes
            quote2 = (not quote2)
            continue
        if st[ix] == "'" and (not quote2) and (backs%2 == 0): #close single quotes
            quote = (not quote)
            continue
    return result

newFiles = False #new files created

if OVERWRITE or (not os.path.exists("./page.pickle")): #if parsed data file doesn't exist (or OVERWRITE is manually set), parse Wikipedia dump file and create it
  idToWord = {} #dict to get string from int id
  wordToId = {} #dict to get int id from string
  idToRootId = {} #dict to get single int id for words with possible several ids
  with open("./" + wiki + "-" + version + "-page.sql", "r", errors = "ignore") as f:
    for line in tqdm(f):
        if len(line) > 500: #"small" lines of dump file contain metadata, "big" lines contain lists of required items
            items = splitLineIntoItems(line)
            for item in items: #use values from items to get int ID and WORD string
                idToWord[int(item[0])] = item[2]
                wordToId[item[2]] = int(item[0])
  for wordId in idToWord: #make single ids for words with severals ids
    idToRootId[wordId] = wordToId[idToWord[wordId]]
  with open("./page.pickle","wb") as f: #save data to file
    pickle.dump((idToWord, wordToId, idToRootId), f)
  !rm {wiki}-{version}-page.sql
  gc.collect()
  newFiles = True
else: #if file exists, read the data
  with open("./page.pickle","rb") as f:
    idToWord, wordToId, idToRootId = pickle.load(f)

if OVERWRITE or (not os.path.exists("./categorylinks.pickle")): #if parsed data file doesn't exist (or OVERWRITE is manually set), parse Wikipedia dump file and create it
  parents = {} #dict to get parents for category id
  sons = {} #dict to get sons for category id
  with open("./" + wiki + "-" + version + "-categorylinks.sql", "r", errors = 'ignore') as f:
    for line in tqdm(f):
        if len(line) > 500: #"small" lines of dump file contain metadata, "big" lines contain lists of required items
          items = splitLineIntoItems(line)
          for item in items: #use values from items to get parent-son category link
            if (item[1] in wordToId) and (int(item[0]) in idToRootId):
              son = idToRootId[int(item[0])]
              parent = wordToId[item[1]]
              parent = idToRootId[parent]
              sons.setdefault(parent, [])
              sons[parent].append(son)
              parents.setdefault(son, [])
              parents[son].append(parent)
    with open("./categorylinks.pickle", "wb") as f: #save data to file
      pickle.dump((parents, sons), f)
    !rm {wiki}-{version}-categorylinks.sql
  gc.collect()
  newFiles = True
else: #if file exists, read the data
  with open("./categorylinks.pickle","rb") as f:
      parents, sons = pickle.load(f)

if newFiles == True and USED_GOOGLE_DRIVE_DIR != "":
  !cp ./*.* {USED_GOOGLE_DRIVE_DIR}/

1066it [07:11,  2.47it/s]
4356it [27:37,  2.63it/s]


CPU times: user 33min 58s, sys: 13.8 s, total: 34min 12s
Wall time: 35min 23s


Служебные категории Википедии объединяют страницы и категории по техническим признакам Википедии и поэтому они исключаются из рассмотрения.
___

Wikipedia administration categories unite pages and categories by Wikipedia technical features and therefore they are excluded from consideration.

In [4]:
banned = set() #banned categories to be skipped in text analysis
#ban sons of main administrative Wikipedia categories
toBan = set([wordToId["'Скрытые_категории'"], wordToId["'Википедия:Служебные'"], wordToId["'Википедия:Участники_по_языкам'"]])
for wordId in idToWord:
  if wordId in parents:
    for word in toBan:
      if word in parents[wordId]:
        banned.add(wordId)
#ban administrative Wikipedia categories, containing specific words
toBan = ["Википедия:", "Проекты:", "'Проект:", "Шаблон", "шаблон", "Портал", "портал", "Страницы_значений", "Перенаправления", "Категории", "Статьи"]
for word in wordToId:
  if wordToId[word] in banned:
    continue
  for ban in toBan:
    if ban in word:
      banned.add(wordToId[word])
#ban some of administrative Wikipedia categories
banned.add(wordToId["'10_000_важнейших_статей'"])
banned.add(wordToId["'Терминология'"])
banned.add(wordToId["'Концепции_по_типу'"])

После полученных пар родительских и дочерних категорий можно извлекать список родителей и потомков до заданного уровня для поиска обобщающих категорий.

____

When parent and son category pairs are obtained, list of ancestors and descendants up to specified level can be found to define common categories.

In [5]:
def getParentsListRec(word, levelNow, resultLevels, used, result, verb = 0):
  #Get list of parent categories for a word in specified levels from a list
  #recursive function to be launched from the main function
  global wordToId, idToWord, banned #global Wikipedia category relationship information
  if levelNow == resultLevels[-1] + 1 or levelNow >= 30: #stop if recursion is too deep for required levels
    return
  if not word in wordToId: #stop if no information is available for this word
    return
  ID = wordToId[word]
  if verb == 2 or (verb == 1 and levelNow in resultLevels): #print the word if required
      print(word)
  if levelNow in resultLevels: #if it's required level, add this word to result
      result.add(word)
  if not ID in parents: #stop if this word has no parent categories
      return
  if not levelNow == resultLevels[-1]:
    for p in parents[ID]: #check all parent categories, launch recursion for new and allowed
        if not p in banned:
            if not p in used:
                used.add(p)
                getParentsListRec(idToWord[p], levelNow+1, resultLevels, used, result, verb)

def getSonsListRec(word, levelNow, resultLevels, used, result, verb = 0):
  #Get list of son categories for a word in specified levels from a list
  #recursive function to be launched from the main function
  global wordToId, idToWord, banned #global Wikipedia category relationship information
  if levelNow == resultLevels[-1] + 1 or levelNow >= 30: #stop if recursion is too deep for required levels
      return
  if not word in wordToId: #stop if no information is available for this word
      return
  ID = wordToId[word]
  if verb==2 or (verb==1 and levelNow in resultLevels): #print the word if required
      print(word)
  if levelNow in resultLevels: #stop if no information is available for this word
      result.add(word)
  if not ID in sons: #stop if this word has no son categories
      return
  if not levelNow == resultLevels[-1]:
    for s in sons[ID]: #check all son categories, launch recursion for new and allowed
        if not s in banned:
            if not s in used:
                used.add(s)
                getSonsListRec(idToWord[s], levelNow+1, resultLevels, used, result, verb)

def getParentsList(word, resultLevels=[1], verb=0):
  #Get list of parent categories for a word in specified levels from a list
  #main function
  used = set()
  result = set()
  resultLevels = sorted(resultLevels)
  getParentsListRec(word, 0, resultLevels, used, result, verb) #run recursion from level 0
  return result

def getSonsList(word, resultLevels=[1], verb=0):
  #Get list of son categories for a word in specified levels from a list
  #main function
  used = set()
  result = set()
  resultLevels = sorted(resultLevels)
  getSonsListRec(word, 0, resultLevels, used, result, verb) #run recursion from level 0
  return result

print(getSonsList("'Китайская_кухня'", [2], verb=0))

{"'Шаосинское_вино'", "'Чоу_мейн'", "'Вок'", "'Баоцзы'", "'Шанхайская_кухня'", "'Кокосовый_джем'", "'Лагман'", "'Tremella_fuciformis'", "'Рисовый_пудинг'", "'Цыплёнок_генерала_Цо'", "'Ютяо'", "'Курица_в_коле'", "'Мога_(грибы)'", "'Китайские_алкогольные_напитки'", "'Жареный_рис'", "'Палочки_для_еды'", "'Сяочи'", "'Китайская_лапша'", "'Гунбао'", "'Вонючий_тофу'", "'Сычуаньская_кухня'", "'Креветочные_шарики'", "'Маньчжурская_кухня'", "'Пекинская_утка'", "'Муравьи_взбираются_на_дерево'", "'Пакистано-китайская_кухня'", "'Няньгао'", "'Дамплинги'", "'Кантонская_кухня'", "'Ашлян-фу'", "'Шаобин'", "'Хойсин_(соус)'", "'Соевая_паста'", "'Хого'", "'Спринг-роллы'", "'Димсам'", "'Соевый_соус'", "'Съедобные_водоросли'", "'Далеба'", "'Цзяоцзы'", "'Хризантемовый_чай'", "'Китайский_чай'", "'Вонтоны'", "'Танъюань_(блюдо)'", "'Макаронные_изделия'", "'Лудагунь'", "'Маринованные_яйца'", "'Тунцзыдань'", "'Маньтоу'", "'Рыбные_шарики'", "'Цзунцзы'", "'Арахисовый_соус'", "'Гобаожоу'", "'Пекинская_кухня'", "'Сто

## 1) ЛИШНИЕ СЛОВА / EXTRA WORDS

Вещи и их свойства могут изучаться через их сопоставления и выявление сходств и различий. Любые две и более вещи имеют общие и различные свойства. Для объяснения различия в "поведении" (y) нескольких вещей требуется как минимум выявить свойства (X) (статические, динамические, контекстные и любые другие), которые у них отличаются, которые могли бы объяснить различие в "результате". <br>
В данной работе осуществляется поиск лишнего слова из рядов слов с "объяснением" через нахождение общих и различных категории Википедии.
___

Things and their features can be investigated by comparing these things with each other to find common and different properties. Any two or more things have common and different features. To explain different "behaviour" (y) of several things, first of all different features (X) (static, dynamic, context and other) should be found. Differencies in features can be used in explanation of differencies in their "result". <br>
In this work, the extra word is estimated for each set of words with "explanation" by finding common and different Wikipedia categoires of these words.

### 1.1 Подготовка данных / Data preparation

Для уточнения наиболее подходящей страницы Википедии к каждому слову (для получения корректных категорий) используется библиотека "wikipedia".
___
To specify the most suitable Wikipedia pages to each word (for a better category extraction) "wikipedia" Python library is used.

In [6]:
#install library to get possible Wikipedia article names from a word
!pip install wikipedia
import wikipedia
wikipedia.set_lang("ru")
print()
print(wikipedia.search("Огурец")[0])

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=726b92c30fe07a7ac6a691b0e787fce9b22d9fceafba581ce522640ebbff3e9e
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia

Огурец обыкновенный


Для большинства слов корректно определяется правильная страница Википедии, но в некоторых случаях страница определяется неверно и это является проблемой, которую можно будет решить в будущей работе. Для слов с неправильным определением страницы была произведена замена на более конкретные слова с корректным определением.
___

For the most of words Wikipedia pages are found correctly, but in some cases not, which is a problem that can be solved in the future work. Words with wrong pages were substituted to more specific ones.

In [7]:
wordLists=[] #lists of words
extraWords=[] #possible extra words for each word list

def add(st): #split, transform and add list of word lists
  global wordLists
  wordLists.append(st.lower().split(", "))

#word lists to be checked, correct answers are marked by "*"
add("платье, свитер, шапка*, рубашка")
add("берёза, дуб, сосна*, земляника*")
add("книга*, телевизор, радиоприёмник, магнитофон") #книга, телевизор, радио, магнитофон
add("чайник, кружка, колбаса*, кастрюля")
add("кепи, шапка, шляпа, тапочки*") #кепка, шапка, шляпа, тапок
add("час, минута, лето*, секунда")
add("ложка, тарелка, кастрюля, сумка*")
add("грузовой_автомобиль, мотороллер*, автомобиль, велосипед*") #грузовик, скутер, автомобиль, велосипед
add("диван, шкаф, стол, лампа_накаливания*") #диван, шкаф, стол, лампа
add("километр, миля, миллиметр, дюйм, килограмм*")
add("самолёт, вертолёт, ворон, шмели, поезд*") #самолёт, вертолёт, ворон, шмель, поезд
add("морковь_посевная, картофель, огурец_обыкновенный, яблоко*") #морковь, картофель, огурец, яблоко
add("сосиска, печенье, тарелка*, сыр")
add("яблоко, слива, груша, огурец_обыкновенный*") #яблоко, слива, груша, огурец
add("молоко, творог, сметана, хлеб*")
add("перчатки*, ботинки, сапоги, туфли")
add("мухи, домовый_воробей*, стрекозы, зелёный_кузнечик") #муха, воробей, стрекоза, кузнечик
add("мандарин, банан, томат*, лимон") #мандарин, банан, помидор, лимон
add("автомобиль, троллейбус, самолёт, скакалка*") #машина, троллейбус, самолёт, скакалка
add("синицы*, индейки, гуси, петух") #синица, индюк, гусь, петух
add("пенал_(принадлежность), тетрадь, карандаш, волчок_(игрушка)*") #пенал, тетрадь, карандаш, юла
add("обыкновенный_сом, щука, жесткокрылые*, Речной_окунь") #сом, щука, жук, окунь"
add("куртка, полотенце*, платье, костюм")

for wordList in wordLists: #process each word list
  extraWords.append([])
  for i in range(len(wordList)): #process each word in the list
    bExtra = False
    if "*" in wordList[i]: #remove "*" char and save information
      wordList[i] = wordList[i][:-1]
      bExtra = True
    wordList[i] = "'" + wordList[i][0].upper() + wordList[i][1:] + "'" #transform the word into the common format
    if not wordList[i] in wordToId: #if the word is not in dictionary, find anything appropriate from Wikipedia and replace
      print(wordList[i],'is missing')
      wordList[i] = "'" + wikipedia.search(wordList[i])[0].replace(" ", "_") + "'"
      print('Replaced by', wordList[i])
    if bExtra: #save extra words
      extraWords[-1].append(wordList[i])

### 1.2 Функции для выявления лишних слов по категориям / Functions for the "extra" words search via category information

Главная суть алгоритма в поиске **категории**, в которую входят **все** слова **кроме** "лишнего", которая объясняла бы **почему** это слово "лишнее". В ходе работы были сделаны доработки алгоритма для повышения "качества" найденных категорий и отсева ситуаций где слова не были занесены или были занесены в категорию по ошибке или без большого значения.

Для каждого потенциального "лишнего" слова ищутся "объясняющие" категории. В первую очередь осуществляется поиск категорий наилучшего "качества", если это не удаётся для всех "лишних" слов, то более "худшего" качества. Ранги "объясняющих", "эксклюзивных" категорий:

1) Родительская категория, которая есть у всех слов, но отсутствует у "лишнего" слова. Пример: "лишнее" слово "Роза" не входит в категорию "Дерево", остальные - "Дуб", "Берёза" - входят.

2) Родительская категория только у остальных слов, но входящяя в родительскую категорию общую для всех слов, включяя "лишнее" (для свидетельства что эта "объясняющая" категория соотвествует общей "тематике" и актуальна). Пример: "лишнее" слово "Роза" не входит в категорию "Дерево", остальные - "Дуб", "Берёза" - входят, "Дерево" как и все слова из списка принадлежат категории "Растения".

3) Родительская категория только у остальных слов, но входящяя в родительскую категорию общую для всех слов и при этом у "лишнего" слова есть "альтернативная" категория - имеющая ту же родительскую категорию и не имеющая пересечение (через дочерние категории) с этой (для свидетельства осознанного отсутствия занесения "лишнего" слова в эту "эксклюзивную" категорию). Пример: "Роза" входит в категорию "Цветы", которая является "альтернативой" категории "Деревья", которые имеют общую категорию "Растения" и не имеют пересечений в потомках, "Дуб" и "Берёза" имеют категорию "Дерево".
___

The core essense of the algorithm is to find a **category**, that contains **all** the words, **except** the "extra" word, which would explain **why** is this word "extra". During the work, some algorithm improvements were done to increase the "quaity" of found categories and to filter out some of cases when words are not included or are included in a category by mistake, or this including is not very significant.

For every potential "extra" word the algorithm tries to find "explaining" categories. First of all, the algorithm tries to find the "best" "explaining" categories, if it fails for every possible "extra" word, the "quality" is reduced and the search is repeated again. Ranks of "explaining" categories:

1) Parent category, which includes all the words except the "extra" word. Example: the "extra" word "Rose" is not included into "Trees" category, all the rest - "Oak", "Birch" - are included.

2) Parent category, which includes all the words except the "extra" word and is included itself in a category, that is common for all the words with the "extra" words (as some indirect evidence that this "explaining" category corresponds to the common "topic"). Example: "extra" word "Rose" is not included into "Trees" category, all the rest - "Oak", "Birch" - are included, "Trees" as all the words from the list is included into "Plants" category.

3) Parent category with all the words except the "extra" word included in a common category, when the "extra" word has an "alternative" category, which has the same parent categories and has no interception (in son categories) with the "explaining" category (as indirect evidence of intentional absence of "extra" word in this "explaining" category). Example: the "extra" word "Rose" is included into "Flowers" category, that is alternative to "Trees" category, which have common "Plants" category and have no intersection in son categories, "Oak" and "Birch" are included into "Trees" category.


In [8]:
def getCommonSonsDistance(words, limit=5, verb=0):
  #Find distance to the closest common son category for a list of words
  for depth in range(1, limit + 1):
    common = set()
    for i in range(len(words)):
        result = getSonsList(words[i], range(0, depth + 1)) #get sons for this word
        if i == 0: #the first set of sons is used for intersection with others
            common = result.copy()
        else:
            common = set.intersection(common, result)
    if len(common) > 0: #if common sons are found at this depth, return results
      if verb > 0:
        print(common)
      return depth, common
  return 100, {}

def getExtrasWithAlternatives(exclusiveCommonParents, extraParents, SEARCH_QUALITY):
  #Having exclusive common categories, find only those, which have mutually exclusive alternatives among the extra word parents, as some evidence of not just missing this category in the extra word
  if SEARCH_QUALITY > 1: #if the search quality is low, don't filter out categories
    return "остальное - " + str(exclusiveCommonParents)
  result = ""
  for cat in exclusiveCommonParents: #check each category for mutually exclusive sons
    proofParents = getParentsList(cat, [1]) #get 1-parents of possible "proof" categoires
    commonMetaParents = set.intersection(proofParents, extraParents) #get common parents for "proof" and extra word categories parents
    if len(commonMetaParents) == 0: #if these sets have no common parents
      #there is no any evidence, that this exclusive category is not just missing in the extra word categories by mistake
      continue
    resultExtraCats = ""
    for metaParent in commonMetaParents: #for each common metaparent find its sons and check the extra word exclusive categories from them
      brotherAlternatives = getSonsList(metaParent, [1])
      brotherAlternatives = set.intersection(brotherAlternatives, extraParents)
      filteredBrotherAlternatives = []
      for extraParentAlternative in brotherAlternatives: #check is this category mutually exclusive with the current common words category
        if SEARCH_QUALITY == 0: #for "high-quality" search mutual exclusivity should be more strict
          depth = 5
        if SEARCH_QUALITY == 1:
          depth = 2
        if getCommonSonsDistance([cat, extraParentAlternative], depth+1)[0] >= depth: #if these alternative categories have no close common sons, this is some evidence of mutual exclusivity
          filteredBrotherAlternatives.append(extraParentAlternative)
      if len(filteredBrotherAlternatives) != 0: #if something was found - add it to results
        resultExtraCats += str(filteredBrotherAlternatives) + ";"
    if resultExtraCats != "":
      result += "не " + cat + ", но в " + str(metaParent) + " как " + resultExtraCats
      break
  return result

def findExtraWordFromList(words, trueExtraWords):
  #Having list of words - find possible extra words and calculate precision corresponding to correct answers
  stResult = " ".join(words) + " => " #put the word list to results to be printed

  for SEARCH_QUALITY in range(5): #try to find "high-quality" "proof" that some word is "extra", in case of fail reduce the quality and try again
    catsForExtraWords = {} #found extra words and categories as "proof"
    extraFound = False
    if SEARCH_QUALITY < 4: #if nothing was found for several attempts, try to find deeper parent-categories, which can be often too abstact
      depthLevels = 5
    else:
      depthLevels = 9

    if SEARCH_QUALITY < 3: #try to find "proof" categories in a common "domain" of caterories (including the extra word), to be more "convincing"
      commonParents = set()
      allowedWords = set()
      for i in range(len(words)): #get parents for each word and find intersection
        result = getParentsList(words[i], range(1, depthLevels+3))
        if i == 0:
          commonParents = result.copy()
        else:
          commonParents = set.intersection(commonParents, result)
      for p in commonParents: #find sons of common parents to define allowed "domain" for "proof" categories
        result = getSonsList(p, range(1, depthLevels+1))
        allowedWords.update(result) #unite sons into allowed "domain"

    for depth in range(1, depthLevels+1): #try to find extra words starting from closest parents as "proof" categories
      if extraFound: #if extra word was found, stop searching in deeper levels
        break
      for extra in range(len(words)): #try each word as "extra"
        commonParents = set() #common parent categories for all the words from the list, except the extra word
        for i in range(len(words)): #get intersection of parents for words excluding the extra word
          if i == extra:
            continue
          result = getParentsList(words[i],range(1,depth+1))
          if i == 0 or (extra == 0 and i == 1):
              commonParents = result.copy()
          else:
              commonParents = set.intersection(commonParents, result)
        result = getParentsList(words[extra], range(1, depth+3)) #get parent categories for the extra word
        extraParents = result.copy()
        exclusiveCommonParents = set.difference(commonParents, result) #find common categories, which are not parent for the extra word and can be considered as "proof" categories
        if SEARCH_QUALITY < 3: #remove categories which are out of the allowed "domain"(which have no common close parents with the extra word parents)
          exclusiveCommonParents = set.intersection(exclusiveCommonParents, allowedWords)
        if len(exclusiveCommonParents) > 0 and len(result) > 0: #if some "proof" categories were found
          if not words[extra] in catsForExtraWords:
            catsForExtraWords[words[extra]] = []
          #for "high-quality" search remove all "proof" categories which have no "brother" alternative categories in the extra word parents
          catsForExtraWords[words[extra]].append( getExtrasWithAlternatives(exclusiveCommonParents, extraParents, SEARCH_QUALITY) )
          if catsForExtraWords[words[extra]][-1] == "": #if all the "proof" categories were filtered out - remove this list
            catsForExtraWords[words[extra]] = catsForExtraWords[words[extra]][:-1]
          if len(catsForExtraWords[words[extra]]) > 0:
            extraFound = True
    if extraFound: #print the result
      print(stResult)
      break
  if not catsForExtraWords: #if nothing was found just print the list of words
    print(stResult)
  #calculate metrics for the result
  iTotal = 0
  iCorrect = 0
  for extraWord in catsForExtraWords: #check all found extra words
    if len(catsForExtraWords[extraWord])>0:
      iTotal += 1 #get total amount of answers
      if extraWord in trueExtraWords:
        iCorrect += 1 #get total amount of answers which are correct
      print(extraWord, str( catsForExtraWords[extraWord]).replace("\\'","'") )
  if iTotal == 0: #no answers means that nothing is solved
    precision = 0
  else:
    precision = 100. * iCorrect / iTotal #get the ratio of correct answers
  print(str( int(precision) ) + "%")
  return precision

Наборы слов для теста были выбран произвольно (в этом проекте демонстрируются пример работы с категориями Википедии и не ставится цель что-то доказать и достоверно сравнить). В качестве метрики используется соотношение правильных ответов к общему числу ответов. Правильные ответы были выбраны предварительно вручную и к ним добавились те, которые были выбраны алгоритмом с корректным объяснением почему это слово можно считать "лишним".
___

Word sets were selected arbitrary (this project doesn't attemp to prove anything or compare results with scientific methology, but it only demonstrates how Wikipedia categories can be used). To estimate results, the ratio of correct answers to the total amount of answers was used. Correct answers were selected manually before testing and some other words, chosen by the algorithm with proper explanations of being "extra", were added to the correct answers.

### 1.3 Результаты / Results

In [9]:
precision = 0
for i, wordList in enumerate(wordLists):
  print(i + 1)
  precision += findExtraWordFromList(wordList, extraWords[i])
  print()
print("Precision", str( int(precision / len(wordLists)) ) + "%")

1
'Платье' 'Свитер' 'Шапка' 'Рубашка' => 
'Шапка' ['не 'Плечевые_изделия', но в 'Одежда_по_расположению_на_частях_тела' как ["'Головные_уборы'"];']
100%

2
'Берёза' 'Дуб' 'Сосна' 'Земляника' => 
'Сосна' ['не 'Цветковые', но в 'Высшие_растения' как ["'Голосеменные'"];']
100%

3
'Книга' 'Телевизор' 'Радиоприёмник' 'Магнитофон' => 
'Радиоприёмник' ['не 'Потребительские_товары', но в 'Продукция_по_видам' как ["'Продукты_умственного_труда'", "'Военная_продукция'", "'Детали_машин_и_механизмов'"];']
0%

4
'Чайник' 'Кружка' 'Колбаса' 'Кастрюля' => 
'Колбаса' ['не 'Кухонное_оборудование', но в 'Гастрономия' как ["'Пища'", "'Кулинария'"];']
100%

5
'Кепи' 'Шапка' 'Шляпа' 'Тапочки' => 
'Тапочки' ['не 'Головные_уборы', но в 'Одежда_по_расположению_на_частях_тела' как ["'Обувь'"];']
100%

6
'Час' 'Минута' 'Лето' 'Секунда' => 
'Лето' ['не 'Время', но в 'Физические_величины_по_алфавиту' как ["'Температура'"];["'Земля'", "'Природные_явления'"];["'Температура'"];']
100%

7
'Ложка' 'Тарелка' 'Кастрюля' 

Выводы

Разработанный алгоритм позволяет определять лишнее слово в большинстве рядов слов, предоставляя корректное объяснение отличия в большинстве случаев. Но результаты не идеальны и сами категории Википедии далеко не всегда содержат "очевидные" связи слов. Также для некоторых отдельных аспектов вещей, таких как цвет, вкус и другие, нет и практически нецелесообразны объединения в категории. Некоторые слова имеют мало категорий, в то время как другие содержат изыточное количество категорий практически не имеющих к ним прямого отношения. Некоторые "очевидные" категории могут находится на большой дистанции, в то время как "абстрактные" и "глобальные" могут оказаться на близкой дистанции. Многие категории являются служебными и не всегда легко (алгоритмически) отличимы от смысловых категорий. Не всегда можно выявить однозначное соответствие слова и страницы в Википедии. При работе с категориями Википедии нужно учитывать эти проблемы.
___

Conclusions

The developed algorithm allows to find the "extra" word in the most of word sets with correct explanations in the most cases. But the results are not perfect and Wikipedia categories themselves not always have "obvious" relations of words. Also, for some separate aspects of things like color, taste and other, there are no categories and they would be practically useless. Some words have very few categories, when other have too many categories almost without direct relation to them. Some "obvoius" categories are far away from words, while "abstract" and "global" categories can be closer. Many categories are Wikipedia administrative categories and it is not always easy (by an algorithm) to separate them from meaningful categorires. It is no always possible to find the correct Wikipedia page for a word. All these problems should be taken into account in a work with Wikipedia categories.

## 2) АНАЛИЗ АВТОРОВ И ТЕКСТОВ ЧЕРЕЗ КАТЕГОРИИ / AUTHORS AND TEXTS ANALISIS VIA CATEGORIES

Частотный анализ слов позволяет получить сжатое представление о тексте (для оценки, подготовки к чтению, использованию в классификации текстов), не читая его целиком, или даже выявить глобальные закономерности неочевидные при чтении. Но одни и те же темы могут описываться разными близкими по тематике словами и для более гибкой группировки слов в темы можно воспользоваться категориями Википедии. <br>
В данной работе осущеcтвляется анализ текстов 5 поэтов для выявления главных категорий каждого автора и всех текстов.
___

Word frequency analysis allows to get compressed presentation about a text (for examination, preparation for reading, text classification) without reading the whole text, or even allows to find some global patterns which are not obvious during reading. But the same topics can be describes by different related words and Wikipedia categorires can be used for agile aggregation of words into topics. <br>
In this work, an analysis of 5 poets' texts is conducted to identify the main categories of each author and all texts.

### 2.1 Подготовка данных / Data preparation

Данные по частотам слов в русском языке взяты с сайта http://dict.ruslang.ru/freq_faq.html <br>
Стихи Пушкина, Есенина, Тютчева, Блока и Маяковского взяты из проекта https://github.com/sberbank-ai/classic-ai
___

Word frequencies in Russian are taken from the site http://dict.ruslang.ru/freq_faq.html <br>
Poems of Pushkin, Yesenin, Tyutchev, Blok and Mayakovsky are taken from the project https://github.com/sberbank-ai/classic-ai

In [10]:
#http://dict.ruslang.ru/freq_faq.html
if not os.path.exists("./freq.csv"):
  !wget http://dict.ruslang.ru/Freq2011.zip
  !unzip Freq2011.zip && rm Freq2011.zip
  !mv ./freqrnc2011.csv ./freq.csv
#https://github.com/sberbank-ai/classic-ai
if not os.path.exists("./classic_poems.json"):
  !wget https://raw.githubusercontent.com/sberbank-ai/classic-ai/master/data/classic_poems.json

--2024-02-21 08:19:52--  http://dict.ruslang.ru/Freq2011.zip
Resolving dict.ruslang.ru (dict.ruslang.ru)... 83.149.210.98
Connecting to dict.ruslang.ru (dict.ruslang.ru)|83.149.210.98|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 493491 (482K) [application/zip]
Saving to: ‘Freq2011.zip’

Freq2011.zip        100%[===================>] 481.92K   708KB/s    in 0.7s    

2024-02-21 08:19:53 (708 KB/s) - ‘Freq2011.zip’ saved [493491/493491]

Archive:  Freq2011.zip
  inflating: freqrnc2011.csv         
  inflating: freqrnc_readme.txt      
--2024-02-21 08:19:54--  https://raw.githubusercontent.com/sberbank-ai/classic-ai/master/data/classic_poems.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3966534 (3.8M) [text/plain]
Saving to: ‘

На основе словаря частот слов в русском языке вычисляется словарь "частот" категорий Викпедии для русского языка (freqs -> globalWikiFreqs).

По датасету текстов поэтов вычисляется словарь частот слов для каждого поэта и для всех поэтов суммарно. На их основе вычисляются словари "частот" категорий Википедии для каждого поэта и всех поэтов (dataset -> freqsByAuthor + globalAuthorFreqs -> wikiFreqsByAuthor + globalWikiAuthorFreqs).
___

The frequency dictionary of the Russian langage is used to calculate a "frequency" dictionary of Wikipedia categories (freqs -> globalWikiFreqs).

Frequency dictionaries for each poet and all poets are calculated from the poet dataset. These dictionaries are used to calculate "frequency" dictionaries of Wikipedia categories for each poet and all poets (dataset -> freqsByAuthor + globalAuthorFreqs -> wikiFreqsByAuthor + globalWikiAuthorFreqs).

При расчёте "частот" категорий Википедии каждое слово из частотного словаря увеличивает "частоту" категорий предков до заданного уровня на своё количество.
___
In calculation of Wikipeda categories "frequency" dictionary each word from frequency dictionary increases "frequency" of ancestor categories up to specified level by its amount of occurences.

In [11]:
def getWikiCatsFreqEstimate(wordsCount, LEVELS = 5):
  #Get rough approximate wiki categories counts from raw word counts.
  #Every raw word increases it's <LEVEL>-parents counts by its count
  global idToWord, wordToId
  countsRes = {}
  iters = 0
  for wrd in tqdm(wordsCount):
      iters += 1
      if iters % 500 == 0: #clean RAM every 500 words
          gc.collect()
      if (not (wrd in wordToId)): #if the word isn't in wiki dictionary it can't have parents and should be used alone
          countsRes[wrd] = wordsCount[wrd]
          continue
      else:
          countsRes.setdefault(wordToId[wrd],0) #create count for the new word
          countsRes[wordToId[wrd]] += wordsCount[wrd]
      result = getParentsList(wrd, range(1, LEVELS+1), 0) #get all parents up to LEVEL
      toAdd = wordsCount[wrd]
      for wrd2 in result: #increase count for each parent
          if wrd != wrd2:
              countsRes.setdefault(wordToId[wrd2],0) #create count for the new word
              countsRes[ wordToId[wrd2] ] += toAdd
  return countsRes

In [12]:
#get raw words counts and Wiki categroies counts from language word frequencies

newFiles = False

OVERWRITE = False

if OVERWRITE or (not os.path.exists("./globalWikiFreqs.pickle")):
  freqs = {}
  with open('./freq.csv', 'r') as f: #get word counts from language frequency file
    header = True
    for line in f:
      if header: #skip the first line of header
        header = False
        continue
      vals = line.split("\t")
      if vals[1] == "s" or vals[1] == "s.PROP": #get word counts for nouns
        freqs["'" + vals[0][0].upper() + vals[0][1:].lower() + "'"] = int(vals[5])

  globalWikiFreqs = getWikiCatsFreqEstimate(freqs) #run function to get Wiki categories count
  with open("./globalWikiFreqs.pickle", "wb") as f:
    pickle.dump(globalWikiFreqs,f)
  newFiles = True
else:
  with open("./globalWikiFreqs.pickle", "rb") as f:
    globalWikiFreqs = pickle.load(f)

100%|██████████| 24467/24467 [02:08<00:00, 190.27it/s]


In [13]:
%%time

OVERWRITE = False

if OVERWRITE or (not os.path.exists("./poetFreqs.pickle")): #if file doesn't exist or OVERWRITE is manually set, calculate and save frequencies
  !pip install pymorphy2 #library to get word's normal form and part of speech for proper frequency estimation
  import pymorphy2
  morph = pymorphy2.MorphAnalyzer()
  datasetByAuthor = {}
  with open("./classic_poems.json") as jsonFile: #read the file with poems
    data = json.load(jsonFile)
    for d in data:
      datasetByAuthor.setdefault(d['poet_id'], [])
      datasetByAuthor[d['poet_id']].append(d['content']) #add poem to its author list of poems
  gc.collect()

  freqsByAuthor = {} #raw noun frequencies for each author
  wikiFreqsByAuthor = {} #Wikipedia categories frequencies estimate for each author
  for poet in datasetByAuthor: #process each author
    #concatenate all poems of the poet
    text = "\n".join(datasetByAuthor[poet])
    freqsByAuthor[poet] = {}
    words = re.sub("[0-9]|\(|\)|\"|!|,|\.|«|»|;|:|@|#|\*|&|:|'|%|\$", "", text).split() #remove inapplicable chars and split text into word list
    print(poet, len(words), 'words')
    for w in tqdm(words): #process each word
      parsing = morph.parse(w)[0] #get the normal form of the word
      wrd = parsing.normal_form
      if parsing.tag.POS != "NOUN": #only nouns are used for frequency calculation
        continue
      wrd = "'" + wrd[0].upper() + wrd[1:].lower() + "'" #transform the word into the common format
      freqsByAuthor[poet].setdefault(wrd, 0)
      freqsByAuthor[poet][wrd] += 1 #add 1 to this word amount

    #get Wikipedia categories frequencies estimate for this author from raw word frequencies
    wikiFreqsByAuthor[poet] = getWikiCatsFreqEstimate(freqsByAuthor[poet])
    gc.collect()
  #get global frequencies for authors from summation of two dicts for each author
  globalAuthorFreqs = {}
  globalWikiAuthorFreqs = {}
  for poet in freqsByAuthor:
    for word in freqsByAuthor[poet]:
      globalAuthorFreqs.setdefault(word, 0)
      globalAuthorFreqs[word] += freqsByAuthor[poet][word]
    for word in wikiFreqsByAuthor[poet]:
      globalWikiAuthorFreqs.setdefault(word, 0)
      globalWikiAuthorFreqs[word] += wikiFreqsByAuthor[poet][word]
  with open("./poetFreqs.pickle", "wb") as f: #save frequencies to file
    pickle.dump((globalAuthorFreqs, globalWikiAuthorFreqs, freqsByAuthor, wikiFreqsByAuthor), f)
  newFiles = True
else: #if file exists load frequencies
  with open("./poetFreqs.pickle", "rb") as f:
    globalAuthorFreqs, globalWikiAuthorFreqs, freqsByAuthor, wikiFreqsByAuthor = pickle.load(f)
if newFiles == True and USED_GOOGLE_DRIVE_DIR != "": #if Google Drive is used, copy created files to this directory
  !cp ./*.* {USED_GOOGLE_DRIVE_DIR}/

gc.collect()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 33.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=64683682d5ce7fa8df3f003beece6b70d7185921a5629f523dd870138cee6631
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
pushkin 97711 words


100%|██████████| 4938/4938 [00:23<00:00, 214.04it/s]


esenin 50239 words


100%|██████████| 3636/3636 [00:18<00:00, 201.68it/s]


blok 94400 words


...

100%|██████████| 2414/2414 [00:13<00:00, 184.60it/s]


mayakovskij 38987 words


100%|██████████| 4191/4191 [00:20<00:00, 203.12it/s]


CPU times: user 2min 46s, sys: 853 ms, total: 2min 47s
Wall time: 2min 55s


0

### 2.2 Функции для объединения слов в категории / Functions for grouping words into categories

Дальнейшая обработка текстов авторов заключается в следующем:

1) Частота исходных слов преборазуется в частоту Вики-категорий (без продвижения родительских категорий как в подготовке данных). Слова, которые не имеют категорию с таким названием оставляются как законченные категории.

2) Родительские категории для текущих категорий оцениваются на "пригодность" для объединения. "Лучшие" категории добавляются в список и "поглощают" свои дочерние категории. Процесс объединения повторяется несколько итераций. В конце этого шага каждый автор имеет наибольшие и "лучшие" категории, описывающиие его тексты.

3) Для размещения схожих категорий рядом друг с другом рассчитываются наиболее подгодящие группы по алгоритму, схожему с объединением в категории.
___
Further processing of the texts of the authors is done in the following steps:

1) Raw word frequencies are transfered into Wiki categories counts (without propagation of parent categories as in data preparation). Words that have no Wiki categories are left as final categories themselves.

2) Parents of all current categories are estimated for being "good" to be a union. "Best" categories are added into the list and "absorb" its sons. The process of grouping is repeated in several iterations. In the end of this step every author have biggest and "best" categories describing texts.

3) To put related categories together suitable groups are estimated in a way similar to the categories union in the previous step.

Для многих вещей существуют разделы наук, изучающие их. Это приводит к тому что большинство категорий можно объединить в соответствующие разделы наук (имеющих свои категории) и в данном проекте это мешает выявлению нужных объединений категорий. Поэтому категория "Наука" и ближайшие её дочерние категории были исключены из рассмотрения.
___

Many things has scientific fields that study them. This leads to the fact that most of categories can be united into their corresponding scientific fields (which have their categories). It prevents categories from being united into more suitable categoires for this project. Therefore "Science" categories and its closest descendant categories were excluded from consideration.

In [14]:
science = set()
for r in getSonsList("'Наука'", [1,2,3]):
  science.add(wordToId[r])
banned = set.union(banned, science)

In [15]:
#restore raw author words from ban

for author in freqsByAuthor:
  for w in freqsByAuthor[author]:
    if w in wordToId and wordToId[w] in banned:
      banned.remove(wordToId[w])

In [16]:
def calcAuthorScales():
  #Get total amount of estimated Wikipedia categories usages for each author to use for scaling
  global scales
  scales = {}
  scales["globalPoets"] = 0
  for poet in wikiFreqsByAuthor:
    scales[poet] = 0
    for word in wikiFreqsByAuthor[poet]:
        scales[poet] += wikiFreqsByAuthor[poet][word]
        scales["globalPoets"] += wikiFreqsByAuthor[poet][word]
  return scales

def getStats(w, stats, scales=[], default=0):
  #Get scaled statistics for a word or a word ID, if it exists. Otherwise return default value
  global wordToId, idToWord
  res = default
  if w in wordToId: wId = wordToId[w]
  else:
    if w in idToWord: wId = idToWord[w]
    else: return default
  if wId in stats: res = stats[wId]
  if w in stats: res = stats[w]
  if len(scales) > 1:
    res *= scales[0] / scales[1]
  return res

def getWikiPageFreqFromWordFreq(wordsFreq):
  #Transfer raw word counts into Wikipedia page counts (without parent categories propagation)
  global wordToId, idToWord
  countsRes = {}
  active = set()
  for w in wordsFreq:
    if (not (w in wordToId)): #if this word doesn't exist in Wikipedia pages dict, put it's count for it's string value
      countsRes[w] = wordsFreq[w]
      continue
    else:
      countsRes[ wordToId[w] ] = wordsFreq[w] #if this word exists in Wikipedia dict - put it's count for it's int ID and save as "active" for processing
      active.add(wordToId[w])
  return countsRes, active #return Wikipedia page counts and links to IDs which can be processed further

Алгоритм объединения в категории и формула для оценки "пригодности" категорий

Для избежания повторов и большого числа практически одинаковых категорий выбираются только "наилучшие" категории и "поглощают" дочерние слова и категории, так что аналогичные категории не будут иметь возможности создаться используя те же слова для объединения.

Для предотвращения слишком сильного полгощения слов какой-либо общей абстрактной категорией, используется пороговое значение количества использований входящих слов, после которого категория фиксируется и не участвует в дальнейшем объединении.

Несколько итераций происходит выбор заданного количества "лучших" категорий, которые добавляются в список к исходным и участвуют в дальнейших объединениях.

В формуле оценки "пригодности" категории для объединения используются следующие множители (каждый из которых возводится в указанную в параметрах степень для усиления или ослабления влияния на результат): <br>
**1)** Умножение на количество использований в текстах автора самой категории и дочерних категорий. Количество каждой дочерней категории домножается на количество использования среди всех авторов, возведённое в заданную степень (для повышения важности категорий, используемых всеми авторами). <br>
**2)** Умножение на количество объединяемых слов в текущую категорию (для снижения продвижения категорий с небольшим количеством входящих слов) <br>
**3)** Деление на количество использования в языке (для снижения продвижения слишком общих и малозначащих категорий). <br>
**4)** Деление на количество использования среди всех авторов (для продвижения более специфичных для конкретного автора категорий). <br>
**5)** Умножение на среднее количество использования родительских категорий для текущей категории среди всех авторов (для продвижения "тем" используемых всеми авторами) (может конфликтовать с предыдущим множителем в зависимости от настроек).
___
The algorithm of words unification into categories and the formula of category "suitability" esitmation

To avoid a big amount of very similar categories only "the best" categories are selected and they "absorb" their son words and categories, so that the similar categories can not be assembled from the same words.

To avoid too big absorption of words into some common abstract category, a threshold value for included words count is used, after which a category is fixed and excluded from further unifications.

The following factors are used in the "suitability" formula for categories union (which factor is built to it's degree specified in parameters): <br>
**1)** Multiplication by the amount of occurrences in texts of the author of a category and its son categories. Each amount of son categories is multiplied by it's amount of occurrences in all authors texts built to the specified degree (to emphasize categories used in all authors texts) <br>
**2)** Multiplication by the amount of words united into this category (to prevent propagation of categories with few amount of different words and categories) <br>
**3)** Division by the amount of occurrences in the language (to prevent propagation of too common and nonspecific words)<br>
**4)** Division by the occurrences in texts of other authors (to propagate categories which are more specific for this author) <br>
**5)** Multiplication by the mean occurences of parent categories for this category in all texts of all authors (to propagate "topics" used by all authors) (can be opposite to the previous factor depending on parameters) <br>

In [17]:
def getParentSelectionScore(params, parCat, sonsCount, scales, sons, activeCounts):
  #Get a score for a parent category how "well" it collects and explains currently active son categories
  global wordToId, idToWord
  if sonsCount < 10: #stop calculation for parent categories with very small sons count
    return 0
  global globalWikiFreqs, globalWikiAurhorFreqs, wordToId #global Wikipedia dict and frequencies for the language and the dataset

  WATCH = [] #list of parent categories to be watched for debug and analysis
  bWatch = False
  if parCat in WATCH: #start watching, if this category is in the watch list
    bWatch = True
    print(" === === === ")
    print("watching", parCat)

  #get total language usage of this parent category
  globalTotal = getStats(parCat, globalWikiFreqs) + 10

  #calculate sons raw counts sum and scores sum
  sonsScoreSum = 0 #sons score sum is a score of "how good" is this collection for this parent category and how much should it be united

  for s in sons:
    if s == parCat: #do not handle the same word as its son
      continue
    countAddition = getStats(s, activeCounts)
    scoreAddition = countAddition * (getStats(s, globalWikiAuthorFreqs) + 10) ** (params["SON_POW"]) #if this word is used in the scope of all authors, it is a more suitable word than otherwise
    sonsScoreSum += scoreAddition
    if bWatch: #print debug info for son category if needed
      print("-", idToWord[s], "count", countAddition, "global", getStats(s, globalWikiFreqs), "result", scoreAddition)

  score = (getStats(parCat, activeCounts) + sonsScoreSum) ** (params["COUNT_POW"])
  score *= len(sons) ** (params["SONS_AMOUNT_POW"])
  score /= params["GLOBAL_FREQ_REDUCE_FUNC"](globalTotal) #reduce the score using the category global usage to pick more "specific" categories later
  originality = (getStats(parCat, globalWikiAuthorFreqs) + 10) ** (params["ORIG_POW"]) #small direction to "specific" categories, to define unique topics among other authors. ORIG_POW is a negative value
  score *= originality
  if (parCat in wordToId) and (wordToId[parCat] in parents): #find parents usage and increase its "importance"
    scorePar = 10
    totalPar = 0
    for p2 in parents[wordToId[parCat]]:
      scorePar += getStats(p2, globalWikiAuthorFreqs)
      totalPar += 1
    if totalPar > 1:
      scorePar /= totalPar
    score *= scorePar ** (params["ORIG_PAR_POW"])

  if bWatch: #debug information
    print("cat globalTotal", globalTotal)
    print("its raw count",  getStats(parCat, activeCounts) + sonsCount)
    print("sonsScoreSum", sonsScoreSum)
    print("its authors usage and originality", getStats(parCat, globalWikiAuthorFreqs), originality)
    print("its final result", score)
    print(" === === === ")
  return score

def getExclusiveCatUnionsStep(params, wordsFreq, scales, iCatsToSelect, activeCats, selectedCats):
  #Select specified amount of "best" categories and subtract counts from their sons to prevent the appearance of almost the same categories
  global wordToId, idToWord
  CHECK_PAR_DEPTH = params["CHECK_PAR_DEPTH"]
  verb = params["VERB"]
  parentsCounts = {}
  parentsScores = {}
  parentsSons = {}

  TR = params["THR"] #threshold to prevent big categories from taking part in further unions as they can absorb other categories
  if len(scales) > 1: #scale threshold accordind to the word counts for this author
    TR *= scales[0] / scales[1]

  notActive = set()
  for w in tqdm(activeCats, disable=(params["VERB"]==0)): #check each active category

    if wordsFreq[w] >= TR: #if now this category is too big - stop using it
      notActive.add(w)
      if verb > 0:
        print("Big category is skipped", idToWord[w], wordsFreq[w], TR)
      continue

    parentsList = getParentsList(idToWord[w], range(1, CHECK_PAR_DEPTH + 1)) #get parents for this word and update their stats
    for parCat in parentsList:
      if (parCat == idToWord[w]): continue
      if not parCat in parentsCounts: #init info for the new parent category
        parentsCounts[parCat] = 0
        parentsSons[parCat] = []
      parentsCounts[parCat] += wordsFreq[w]
      parentsSons[parCat].append(w)

  newActive = set()
  for i in range(iCatsToSelect): #select specified number of best categories and subtract counts from their sons
    maxP = None
    for parCat in parentsCounts: #check all obtained parent categories
      parentsScores[parCat] = getParentSelectionScore(params, parCat, parentsCounts[parCat], scales, parentsSons[parCat], wordsFreq) #get the score for this category
      if not (wordToId[parCat] in newActive): #skip category which already was selected recently
        if (maxP is None) or (parentsScores[parCat] > parentsScores[maxP]): #if this score is the biggest, update maxP category to be selected in the end
          maxP = parCat

    if maxP is None: #stop search if nothing can be selected anymore
      if verb > 0:
        print("Nothing to select")
      break

    newActive.add(wordToId[maxP]) #now this category is united from its sons and can be used next time
    if verb > 0:
      print("selected", maxP, parentsCounts[maxP], parentsScores[maxP])
    selectedCats.add(wordToId[maxP])

    for s in parentsSons[maxP]: #process each absorbed son category
      if (idToWord[s] == maxP) or (not (s in activeCats)):
        continue
      sub = wordsFreq[s] #now its absorbed by the parent category, so it is reduced not to be used by similar categories (can be reduced smoothly)
      wordsFreq[s] -= sub
      if wordsFreq[s] < 3: #do not check too small or empty categories anymore
        activeCats.remove(s)
      for p in getParentsList(idToWord[s], range(1, CHECK_PAR_DEPTH + 1)): #find all alternative parents and reduce their counts
        if not (p in parentsCounts): continue
        if p == maxP: continue
        if p == idToWord[s]: continue
        parentsCounts[p] -= sub

    if not wordToId[maxP] in wordsFreq:
      wordsFreq[wordToId[maxP]] = 0
    wordsFreq[wordToId[maxP]] += parentsCounts[maxP] #put the absorbed amount into the category

  return (activeCats.union(newActive)).difference(notActive), selectedCats #update active categores and return with selected categories


Полученные списки содержат категории, которые окалазись "лучше" других в объединении. Далее они сортируются по количеству вхождений скорректированным по формуле для снижения продвижения слишком общих категорий. По заданному параметру "уникальности" отсеиваются категории, которые употребляются не чаще чем во всех текстах всех авторов. Этот параметр позволяет регулировать результаты между - "хорошими", но одинаковыми для всех авторов категориями и "странными", но уникальными для каждого автора.
___
Obtained lists contain categories, which are "better" than other in union. In the next step they are sorted by word count corrected by a formula to reduce propagation of too common categories. In accordance with the "Uniqueness" parameter, categories, which are used not much more often then by all authors, are filtered out. This parameter allows to regulate the results from "good" but common for every author categories and categories that are "strange" but unique for each author.

In [18]:
def getSortedWordList(params, counts, scales, takeTop=500):
  #Get sorted and filtered lists of categories and words according to their "importance" and "uniqueness"
  global globalWikiFreqs, globalWikiAuthorFreqs, wordToId, idToWord

  currentParams = params.copy() #insert default parameters for missing
  defaultParams = {"SORT_UNIQUENESS_DROP_LEVEL" : 3}

  for p in defaultParams:
    if not p in currentParams:
      currentParams[p] = defaultParams[p]

  countsSum = scales[0]
  if len(scales) > 1: globalAuthorSum = scales[1]
  else: globalAuthorSum = countsSum

  topWords = []
  for w in counts:
      score = counts[w]
      score /= params["GLOBAL_FREQ_REDUCE_FUNC"](getStats(w, globalWikiFreqs, scales, 200)) #reduce score for words that are too frequent in the language
      #if word is unique enough add it to the list
      counts[w];
      globalWikiAuthorFreqs[w];
      if counts[w] >= currentParams["SORT_UNIQUENESS_DROP_LEVEL"] * countsSum * globalWikiAuthorFreqs[w] / globalAuthorSum:
        if type(w) == str: topWords.append((score, w, counts[w]))
        else: topWords.append((score, idToWord[w], counts[w]))
  return sorted(topWords)[-min(len(topWords), takeTop):] #get 500 most frequent

def getSortedFreqLists(freqsByAuthor, takeTop=500):
  #Get sorted word counts lists without additional processing to be displayed
  res = {}
  resCommon = {}
  for author in freqsByAuthor: #process each author
    res[author] = []
    resNow = []
    for w in freqsByAuthor[author]:
      resNow.append((freqsByAuthor[author][w], w))
      resCommon.setdefault(w, 0)
      resCommon[w] += freqsByAuthor[author][w]
    resNow = sorted(resNow, reverse=True)
    for r in resNow:
      res[author].append(r[1] + " : " + str(r[0]))
    res[author] = res[author][:min(takeTop, len(res[author]))] #get 500 most frequent

  resCommonList = [(resCommon[w], w) for w in resCommon]
  resCommonList = sorted(resCommonList, reverse=True)
  res["common"] = []
  for r in resCommonList:
    res["common"].append(r[1] + " : " + str(r[0]))
  res["common"] = res["common"][:min(takeTop, len(res["common"]))]
  return res

In [19]:
def calcAuthorCats(scales, params={}):
  #High-level function to obtain categories from raw word frequencies for each author. Unite into several levels of "best" categories
  currentParams = params.copy()
  defaultParams = {"VERB":0, "CATS":40, "LEVELS":2, "THR":150, "CHECK_PAR_DEPTH":1, "CAT_MUL":30,
                   "COUNT_POW":1, "SON_POW":1, "SONS_AMOUNT_POW":1, "ORIG_POW":-0.5, "ORIG_PAR_POW":1.5,
                   "GLOBAL_FREQ_REDUCE_FUNC":np.log2}

  for p in defaultParams:
    if not p in currentParams:
      currentParams[p] = defaultParams[p]

  gc.collect()
  catsForAuthor = {}
  selectedCatsForAuthor = {}

  for author in ["common", *wikiFreqsByAuthor]:
    if currentParams["VERB"] > 0: print(author)
    catsForAuthor[author] = []
    #get initial categories from raw word frequencies
    if author == "common":
      counts, active = getWikiPageFreqFromWordFreq(globalAuthorFreqs)
    else:
      counts, active = getWikiPageFreqFromWordFreq(freqsByAuthor[author])
    if currentParams["VERB"] > 0:
      print(len(active),'active')
    selectedCatsForAuthor[author] = set()

    rawWords = active.copy()

    for i in range(currentParams["LEVELS"]): #run several levels of category unifications
      if author == "common":
        active, selectedCatsForAuthor[author] = getExclusiveCatUnionsStep(currentParams, counts, (scales["globalPoets"],), currentParams["CATS"], active, selectedCatsForAuthor[author])
      else:
        active, selectedCatsForAuthor[author] = getExclusiveCatUnionsStep(currentParams, counts, (scales[author], scales["globalPoets"]), currentParams["CATS"], active, selectedCatsForAuthor[author])
      gc.collect()

    for s in selectedCatsForAuthor[author]: #"emphasize" united categories to be shown before raw words and categories
      counts[s] *= currentParams["CAT_MUL"]

    #sort categories, filter out too common categories, construct text entries for list widgets
    if author == "common":
      for t in reversed(getSortedWordList(currentParams, counts, (scales["globalPoets"],))):
        catsForAuthor[author].append(t[1]+" : "+str(t[2]))
    else:
      for t in reversed(getSortedWordList(currentParams, counts, (scales[author],scales["globalPoets"]))):
        catsForAuthor[author].append(t[1]+" : "+str(t[2]))

  return catsForAuthor, selectedCatsForAuthor

Для размещения рядом друг с другом категорий с общей тематикой для более наглядного представления и сравнения между авторами используется группировка по общим родительским категориям. Как и категории, группы выбираются по максимальному количеству очков по схожей формуле.
___
To place categories of similar topics together for a better presentation and comparison between the authors, a grouping by common parent categories is used. As in categories union, groups are selected by the maximum scores according to a similar formula.

In [20]:
def getParentDepthDistance(words, limit=5, verb=0):
  #Get distance to the closest common parents for words from specified list, also return these parents
  for depth in range(1, limit+1): #check levels
    common = set()
    for i in range(len(words)): #get parents for each word for specified depth and find intersecion
        result = getParentsList(words[i], range(0,depth+1))
        if i == 0: common = result.copy()
        else: common = set.intersection(common, result)
    if len(common) > 0: #if result was found - return it and stop checking deeper levels
      if verb > 0:
        print(common)
      return depth, common
  return 100, {}

def getGroupScore(params, group, wordCounts, score, groupWord, scales):
  #Get a score for a group how "well" it collects and explains it's set of selected categories
  if len(group) <= 1: #it is not a group if it has 1 or 0 members
    return 0
  #get mean sons count
  sonsSum = 0
  for w in group:
    sonsSum += wordCounts[w]
  sonsMeanCount = sonsSum / len(group)
  #get "good" sons count without taking small categories into account
  sonsFilteredCount = 0
  for w in group:
    if wordCounts[w] > sonsMeanCount * params["GROUP_CATS_AMOUNT_FILTER"]:
      sonsFilteredCount += 1
    else:
      sonsFilteredCount += 0.5 * wordCounts[w] / (sonsMeanCount * params["GROUP_CATS_AMOUNT_FILTER"])
  #increase the score by "good" sons amount - it's better to have bigger groups to unite more categories
  score *= sonsFilteredCount ** (params["GROUP_CATS_AMOUNT_POW"])
  #increase the score by total count to select bigger amount of total covered words
  score *= sonsSum ** (params["GROUP_CATS_SUM_POW"])
  #increase or decrese the score with respect to its' global count
  score *= (getStats(groupWord, globalWikiFreqs, scales) + 5) ** (params["GROUP_GLOBAL_FREQ_POW"])
  #increase or decrese the score with respect to its' authors count
  score *= (getStats(groupWord, globalAuthorFreqs, scales) + 2) ** (params["GROUP_AUTHOR_FREQ_POW"])
  if len(group) < params["MIN_GROUP_CATS"]: #unacceptably small groups still can be used if nothing else left
    score /= 100
  return score

def groupSelectedCats(params, cats, selectedCats, scales):
  #Group selected categories to be located together
  global wordToId, idToWord

  currentParams = params.copy() #insert default parameters for missing
  defaultParams = {"GROUPS_TOTAL":16, "GROUP_LEVELS":2, "MIN_GROUP_CATS":3, "CAT_MUL":30, "GROUP_CATS_AMOUNT_POW":0.1, "GROUP_CATS_AMOUNT_FILTER":0.1, "GROUP_CATS_SUM_POW":0.5,
                   "GROUP_GLOBAL_FREQ_POW":0.5, "GROUP_AUTHOR_FREQ_POW":0.7}
  for p in defaultParams:
    if not p in currentParams:
      currentParams[p] = defaultParams[p]

  groups = {}
  groupCounts = {}
  wCount = {}
  for i in range(len(cats)): #check each category
    w1 = cats[i].split(" : ")[0]
    c1 = int(cats[i].split(" : ")[1])
    wCount[w1] = c1
    for j in range(i+1, len(cats)): #check other categories to find close groups
      w2 = cats[j].split(" : ")[0]
      c2 = int(cats[j].split(" : ")[1])
      wCount[w2] = c2
      dist = getParentDepthDistance([w1, w2], currentParams["GROUP_LEVELS"] + 1, 0)
      if dist[0] <= currentParams["GROUP_LEVELS"]: #if these two categories are close
        for gr in list(dist[1]): #build each group and add these categories with their couunts
          if not gr in groups:
            groups[gr] = []
            groupCounts[gr] = 0
          if not w1 in groups[gr]:
            groups[gr].append(w1)
            groupCounts[gr] += c1
          if not w2 in groups[gr]:
            groups[gr].append(w2)
            groupCounts[gr] += c2

  usedCats = set()
  resCats = []
  for iter in range(currentParams["GROUPS_TOTAL"]): #get specified amount of best groups
    maxGroup = None
    for gr in groups:
      if (maxGroup is None):
        maxGroup = gr
        continue
      r1 = getGroupScore(currentParams, groups[maxGroup], wCount, groupCounts[maxGroup], maxGroup, scales)
      r2 = getGroupScore(currentParams, groups[gr], wCount, groupCounts[gr], gr, scales)
      if r2 > r1:
        maxGroup = gr
    if maxGroup is None:
      break
    if getGroupScore(currentParams, groups[maxGroup], wCount, groupCounts[maxGroup], maxGroup, scales) < 5: #stop grouping if the best score is close to zero
      break
    #make the final presentation of the best group
    resCats.append("==" + maxGroup + " : " + str(groupCounts[maxGroup]))
    countedWords = [] #get words from the group with counts to sort
    for w in groups[maxGroup]:
      countedWords.append((wCount[w], w))
      usedCats.add(w)
    for c, w in sorted(countedWords, reverse=True):
      for gr in groups: #check every other group and remove these categories
        if gr != maxGroup:
          if w in groups[gr]:
            groupCounts[gr] -= c
            groups[gr].remove(w)
      if (w in wordToId) and (wordToId[w] in selectedCats): #restore initial score without amplification
        c //= currentParams["CAT_MUL"]
      resCats.append(w + " : " + str(c))
    resCats.append("")
    del groups[maxGroup]

  #put remaining categories without groups
  resCats.append("")
  for i in range(len(cats)):
    w = cats[i].split(" : ")[0]
    c = int(cats[i].split(" : ")[1])
    if w in usedCats:
      continue
    if (w in wordToId) and (wordToId[w] in selectedCats):
      c //= currentParams["CAT_MUL"]
    resCats.append(w + " : " + str(c))

  return resCats

In [21]:
def calcGroups(catsForAuthor, selectedCatsForAuthor, scales, params={}):
  #High-level function to group selected categories to be located together
  groupedCatsForAuthor = {}
  for author in catsForAuthor:
    if author == "common":
      groupedCatsForAuthor["common"] = groupSelectedCats(params, catsForAuthor["common"], selectedCatsForAuthor["common"], (scales["globalPoets"],))
    else:
      groupedCatsForAuthor[author] = groupSelectedCats(params, catsForAuthor[author], selectedCatsForAuthor[author], (scales[author], scales["globalPoets"]))
    gc.collect()
  return groupedCatsForAuthor

In [22]:
def runCatsEstimation(params):
  #Main function to perform calculations of categories and groups
  scales = calcAuthorScales()
  catsForAuthor, selectedCatsForAuthor = calcAuthorCats(scales, params)
  return calcGroups(catsForAuthor, selectedCatsForAuthor, scales, params), catsForAuthor

Полученные категории и группы для каждого автора отображаются в виде интерактивных списков, позволяющих нажать на категорию и получить список слов из текстов автора с их количеством, при нажатии на которые отображается путь категорий от исходного слова до итоговой категории.
___
Obtained categories and groups for each author are displayed in interactive lists, which allow to click on a category to get the list of raw words from texts with their numbers of occurences for this category, that can be clicked to get category paths from the initial word to the category.


In [23]:
def getRawWordsForCat(cat, depth, counts):
    #Get list of raw words with its count for the specified category to be displayed
    res = []
    sonsList = getSonsList(cat, range(1, depth+1))
    for W in counts:
      if (W in sonsList) or (W == cat):
        res.append(W + " : " + str(counts[W]) + "\n")
    return res

def getWordToCatPaths(word, cat, depth=5):
  #Get list of categories on different levels on path from the specified word to the specified category
  domain = getSonsList(cat, range(depth+1) ) #son categories related to this category
  path = {}
  for d in range(depth+1): #init path lists for each level
    path[d] = []
  path[0] = [word]
  used = set()
  #init queue
  queue = [(word, 0)]
  p1 = 0
  p2 = 1
  while (p1 < p2): #run queue
    if queue[p1][1] == depth: #stop if max depth is reached
      break
    parentsList = getParentsList(queue[p1][0], [1])
    for p in parentsList:
      if (not p in domain) or (p == word): #skip word not related to category or the same word
        continue
      if not p in used: #add the new word to the queue to be processed
        used.add(p)
        path[queue[p1][1]+1].append(p) #add the word to its path level
        queue.append((p, queue[p1][1]+1))
        p2 += 1
    p1 += 1
  res = "" #create result based on each depth level of paths
  for d in range(0, depth):
    if len(path[d]) == 0:
      continue
    res += str(d+1) + ": "
    for p in path[d]:
      res += p + " "
    res += "\n"
  return res

In [24]:
class WordPanel(): #interactive categories and word lists
  def __init__(self, catsForAuthor, useDetalization=True):
    #Build columns for categories of each author
    columnWidgets = []

    self.backtrace = widgets.Textarea("", layout=widgets.Layout(height="500px"))

    for author in catsForAuthor:
      title = widgets.Label(author)
      listCats = widgets.Select(options=catsForAuthor[author], layout=widgets.Layout(height="500px"))
      listCats.author = author
      if useDetalization:
        listCats.observe(lambda event, author=author: self.onCategoryClicked(event, author))
      if author != "common":
        columnWidgets.append(widgets.VBox([title, listCats]))
      else:
        columnWidgets.insert(0, widgets.VBox([title, listCats]))

    if useDetalization:
      title = widgets.Label("Cat. raw words")
      listCats = widgets.Select(options=["Press on cat"], layout=widgets.Layout(height="500px"))
      self.catDetails = title
      self.rawWords = listCats
      self.rawWordsTitle = title
      listCats.observe(self.onRawWordClicked)
      columnWidgets.append(widgets.VBox([title, listCats]))
      columnWidgets.append(widgets.VBox([widgets.Label("Word -> cat. paths"), self.backtrace]))

    self.wordPanel = widgets.HBox(columnWidgets)

  def onCategoryClicked(self, event, author):
    #Output raw words from texts for the clicked category
    if event["name"] == "value":
      if (event["new"] == "") or (event["new"][0] == "=") or (not ": " in event["new"]):
        return
      cat = event["new"].split(" : ")[0]
      self.rawWordsTitle.value = author + "," + cat
      self.rawWords.options = ["..."]
      if author != "common":
        self.rawWords.options = getRawWordsForCat(cat, 4, freqsByAuthor[author])
      else:
        self.rawWords.options = getRawWordsForCat(cat, 4, globalAuthorFreqs)

  def onRawWordClicked(self,event):
    #Output categories path from the clicked raw word to it's category
    if event["name"] == "value":
      if not type(event["new"]) == str:
        return
      word = event["new"].split(" : ")[0]
      cat = self.rawWordsTitle.value.split(",")[1]
      if word == "...":
        return
      self.backtrace.value = self.rawWordsTitle.value.split(",")[0] + cat + "\n" + getWordToCatPaths(word,cat)

### 2.3 Выполнение расчётов, интерактивное отображение результатов / Execution, interactive display of results

Далее задаются параметры для всех функций обработки категорий в текстах поэтов, запускается обработка и результат выводится в виде интерактивных списков для каждого поэта. <br>
При нажатии на категорию выводится список слов из текстов автора с их количеством. <br>
При нажатии на исходные слова авторов отображается путь категорий от исходного слова до итоговой категории.
___
Next, the parameters of author categories processing are set. The calculations are executed and the results are displayed with interactive category lists for each author. <br>
A click on a category outputs the list of words from author texts with their numbers of occurences.<br>
A click on a word from this list outputs category paths from this word to this category.

In [27]:
%%time

'''defaultParams = {"VERB":0, "CATS":40, "LEVELS":2, "THR":150, "CHECK_PAR_DEPTH":1,
                   "COUNT_POW":1, "SON_POW":1, "SONS_AMOUNT_POW":1, "ORIG_POW":-0.5, "ORIG_PAR_POW":1.5,
                   "GLOBAL_FREQ_REDUCE_FUNC":np.log2,
                   "GROUPS_TOTAL":16, "GROUP_LEVELS":2, "MIN_GROUP_CATS":3, "CAT_MUL":30, "GROUP_CATS_AMOUNT_POW":0.1, "GROUP_CATS_AMOUNT_FILTER":0.1, "GROUP_CATS_SUM_POW":0.5,
                   "GROUP_GLOBAL_FREQ_POW":0.5, "GROUP_AUTHOR_FREQ_POW":0.7}''';

'''
==PARAMETERS

"VERB":0 - verbosity level
"CATS":40 - amount of categories to be united at one iteration
"LEVELS":2 - amount of iterations for category union. More levels lead to more abstract categories
"THR":150 - threshold to prevent big categories from union and absorbing other categories
"CHECK_PAR_DEPTH":1 - graph distance to common parents for union selection
"CAT_MUL":30 - multiplier to "emphasize" selected categories among frequent raw words

=Score parameters to select "best" unions into bigger categories
"COUNT_POW":1 - exponent for total count to promote categories with higher word coverage
"SON_POW":1 - exponent for son scores among authors texts usage to emphasize "useful" words to be united
"SONS_AMOUNT_POW":1 - exponent for sons amount to promote more diversed categories
"ORIG_POW":-0.5 - exponent for category count among all authors to direct union towards more unique categories
"ORIG_PAR_POW":1.5 - exponent for category parents count to promote categories with frequently used parents (but unique themselves)
"GLOBAL_FREQ_REDUCE_FUNC":np.log2 - function to reduce the score for globally frequent words

=Parameters to unite categories into groups
"GROUPS_TOTAL":16 - amount of groups to unite selected categories
"GROUP_LEVELS":2 - graph parent distance to common parents for union selection into groups
"MIN_GROUP_CATS":3 - minimum amount of categories to create a group
"GROUP_CATS_AMOUNT_POW":0.1 - exponent for category count to promote more diversed groups (small categories are not counted)
"GROUP_CATS_SUM_POW":0.5 - exponent for words count to promote groups with bigger total word coverage
"GROUP_GLOBAL_FREQ_POW":0.5 - exponent for group frequency among language to set importance of frequent usage in language
"GROUP_AUTHOR_FREQ_POW":0.7 - exponent for group frequency among all authors to set importance of frequent usage among authors

"SORT_UNIQUENESS_DROP_LEVEL":3 - how many times count of a word should exceed the average to be left in results (bigger - more specific resuts, smaller - more similar results)
''';




params = {} #use default parameters
groupedCatsForAuthor, catsForAuthor = runCatsEstimation(params)
wp = WordPanel(groupedCatsForAuthor)
display(wp.wordPanel)

CPU times: user 1min 26s, sys: 294 ms, total: 1min 26s
Wall time: 1min 28s


По полученным категориям можно сделать предположение по тематикам, наиболее характерными каждому автору. Например, у Пушкина это могут быть люди, религия, государство, античные персонажи, у Есенина - природа, крестьянский быт, у Маяковского - экономические, социальные и другие термины, актуальные в 20 веке.
Также из полученных категорий можно отметить устаревшие или интересующие слова и их категории в срезе использования авторами (вместо полного или случайного перебора для поиска подобных слов). Теоретически, можно было бы определить крупные категории, соотвествующие времени авторов, но не упоминаемые ими для поиска "новых" возможных тем для текстов.
___
With obtained categories it is possible to make assumptions about main topics for each author. For example, for Pushkin it can be people, religion, Russian Empire, аncient Greek characters, for Esenin - nature, rural peasant life, for Mayakovsky - economic, social and other terms relevant in the 20th century. Also, it is possible to find outdated and interesting words and their categories from author usage projection (Instead of a full or random search for such words). Theoretically, it can be possible to find out big categories, related to authors time, but not used by authors to find some "new" topics for texts.

Частоты слов без объединения в категории / Word counts without category unions

In [26]:
wp = WordPanel(getSortedFreqLists(freqsByAuthor), False)
display(wp.wordPanel)

### 2.4 Результаты / Results

Реализованные функции позволяют выделять категории Википедии из текстов авторов и определять особенности как каждого автора, так и всех текстов. Но для преодоления возникших проблем были созданы механизмы и параметры, подбор которых при длительном времени вычисления может усложнять поиск оптимальных списков категорий. Возможно, такие подходы как машинное обучение могли бы упростить и улучшить качество поиска списков категорий, автоматически выделяя более существенные категории и отсеивая неподходящие связи с родительскими категориями на основе предоставленных признаков, использованных в этой работе.
___
Realized functions allow to extract Wikipedia categories for author texts and to determine main topics for each author and all the texts. But additional algorithms and parameters were created to overcome discovered problems, which, taking into account the duration of the calculations, can complicate the process of finding optimal category lists. Probably, approaches like machine learning can simplify and improve the quality of the category lists search by automated selection of more suitable categories and filtering out inappropriate link to parent categories based on provided features used in this work.

# Выводы и возможная дальнейшая работа / Conclusions and possible further work

-Категориальные связи Википедии заполнены не идеально, содержат много лишних, слишком абстрактных и служебных категорий, имеют пропуски очевидных взаимоотношений. Но при подходящей обработке возникающих проблем, из данной информации можно извлекать корректные сведения для большинства слов. <br>
-Получаемые сведения по категориям для слов могут быть использованы для объединения и разделения слов, выявления общих и различных черт и тематик.
<br> <br>
-В дальнейшей работе можно добавить обработку редиректов, попробовать решить проблему многозначности и отстуствия прямой связи между словом из текста и конкретной страницы Википедии. <br>
-Может быть реализовано машинное обучение на основе векторов слов и категориальных отношений для проверки возможности предсказывать категориальные отношения и уточнения недостающих или лишних категориальных отношений в Википедии. <br>
-Помимо частот использования в языке и текстах можно воспользоваться другими лингвистическими коэффициентами и также рассчитать коэффициенты самостоятельно, разбивая тексты на фрагменты, такие как предложения и абзацы, для выявления специфичности/массовости слов и их релевантности между собой. <br>
-На основе этой информации, а также других алгоритмов, можно "усилить" и "ослабить" как отдельные взаимосвязи категорий в целом, так и релевантность каждой взаимосвязи в каждом конкретном контексте. <br>
-Может быть реализовано определение самых крупных категорий, соответствующих по времени текстам, но не упоминаемых авторами (для определения новых возможных тем). <br>
-Может быть реализован алгоритм для оптимального поиска заданной категории на основе генерируемых вопросов на вхождения в крупные категории для проверки возможности локализации "случайного" понятия. <br>
-Для оптимизации и сокращения времени подбора параметров при объединении категорий может быть реализовано машинное обучение на основе признаков, схожими с теми, что были использованы, и выбранных наиболее "удачных" и "неудачных" объединённых категорий. <br>
-Могут быть использованы обученые языковые модели для получения векторов слов языка, категорий и слов в тексте для уточнения соответствий между ними. <br>
-На основе крупных категорий (в целом или для конкретных наборов текстов) могут быть разработаны признаки для классификации и другой обработки текстов.
___
-Wikipedia categorial relations are not filled perfectly and have many extra, too abstract and administrative categories. A lot of obvious relations are missing. But with a proper processing of these problems it is possible to extract useful information for the most of words. <br>
-Obtained information on words' categories can be used to unite and separate words, determine their common and different features and topics. <br> <br>
-In further work, redirects processing can be added. The problem of ambiguity and a possible absence of a direct link between a word and the right Wikipedia page can be considered. <br>
-Machine learning approach based on word vectors can be used to investigate the possibility to predict categorial relations of words and to find missing or extra Wikipedia categorial relations. <br>
-In addition to word frequencies in the language and texts, it is possible to use other linguistic coefficients and also to calculate new coefficients with datasets split into sentences and paragraphs to estimate how specific or common every word is and how are word related to each other. <br>
-With this information and other algorithms it is possible to "enhance" and "loosen" categorial links in general and in specific contexts. <br>
-It is possible to realize an algorithm to find the biggest categories which correspond to authors' time but are not mentioned in author texts (to find new possible topics). <br>
-Machine learning approach based on features similar to those that were used and manually selected "good" and "bad" united categories can be implemented to optimize and speed up parameters selection for category grouping. <br>
-Trained language models can be used to obtain word vectors for words in general, for categories and for words in texts to specify its relationships between each other. <br>
-Based on the biggest categories (in general or for specific texts), features for text classification and other text processing can be developed.